In [1]:
hf_token = "huggingface-token-here"

In [2]:
from huggingface_hub import notebook_login
notebook_login()

## LLAMA3.1 8B

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import pandas as pd
from tqdm import tqdm
import os
cache_dir = "../cache"
os.environ['TRANSFORMERS_CACHE'] = cache_dir

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda:1", 
    torch_dtype="auto",
    trust_remote_code=True,
    cache_dir=cache_dir, 
    token=hf_token
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
     max_new_tokens=100, do_sample=False, num_return_sequences=1, return_full_text=False, pad_token_id=128009
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=156,
)
print(outputs[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Arrrr, ye landlubber! I be Captain Chatbeard, the scurvy dog o' the seven seas... er, the internet! Me and me trusty keyboard be here to swab the decks o' yer questions and provide ye with the treasure o' knowledge ye seek! So hoist the sails and set course fer a swashbucklin' good time, matey!


In [4]:
tokenizer.eos_token_id

128009

In [5]:
print(pipeline(messages, max_new_tokens=10))

[{'generated_text': 'Arrrr, ye landlubber! I be'}]


In [6]:
# Function to extract sentiment from model output
def extract_sentiment(output):
    if "Positive" in output:
        return "Positive"
    elif "Negative" in output:
        return "Negative"
    elif "Neutral" in output:
        return "Neutral"
    else:
        return "Unknown"


def generate_sentiment(examples):
    outputs =  pipeline(
       examples['prompt'],
        max_new_tokens=5,
                do_sample=False,
        num_return_sequences=1
    )
    return {"sentiment": [output[0]["generated_text"] for output in outputs]}

In [7]:
import datasets
from datasets import Dataset



datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_prompt(example):
    return  [
    {"role": "system", "content": "Following is a tweet extracted from an African American individual's twitter account. Identify the sentiment expressed by the author of the tweet. The sentiment must be classified as either of Positive, Negative, or Neutral."},
    {"role": "user", "content": f"{example['text']}"},
    {"role": "assistant", "content": "The sentiment expressed in this sentence is: "}
]

# Load the dataset
df = pd.read_csv('./seperate models/5000_AAE_sentence.csv')[["text"]]


# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset

dataset = dataset.map(lambda example: {"prompt": create_prompt(example)})

# Generate sentiments
dataset = dataset.map(
    generate_sentiment,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

# # Save the labeled dataset to a CSV file
output_path = './seperate models/LLama3.1-8b-AAE-Labels-FINAL.csv'
labeled_df[["text", "sentiment"]].to_csv(output_path, index=False)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sentiment at 0x7fbc447a1820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


successfully generated models answers


In [265]:
aae_sentiment = pd.read_csv('./seperate models/LLama3.1-8b-AAE-Labels-FINAL.csv')

In [266]:
harmful_removed = aae_sentiment.loc[~sae_labels.index.isin([50, 77, 690, 817, 1010, 1775, 1846, 2941])]

In [267]:
harmful_removed.to_csv('./seperate models/llama-some-removed/LLama3.1-8b-AAE-Labels-FINAL.csv')

____________________

In [8]:
import pandas as pd

In [9]:
dataset = pd.read_csv("./seperate models/5000_AAE_sentence.csv")

## Translation to SAE

In [11]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_prompt(example):
    return  [
    {"role": "system", "content": "Following is a tweet extracted from a African American twitter individual's account. Your task is to convert the tweet to Standard American English. Reply with just the sentence."},
    {"role": "user", "content": f"{example['text']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_prompt(example)})

# Function to generate sentiments
def generate_sae(examples):
    outputs = pipeline(
      examples["prompt"],
        max_new_tokens=100,
                do_sample=False,
        num_return_sequences=1
    )
    return {"standard_american_english": [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_sae,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

# Save the labeled dataset to a CSV file
output_path = './seperate models/Llama-3.1-SAE.csv'
labeled_df[["text", "standard_american_english"]].rename(columns={"text": "african_american_english"}).to_csv(output_path, index=False)

print(f"SAE dataset saved to {output_path}")
print(f"Number of processed sentences: {len(labeled_df)}")


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sae at 0x7fbe617f6ee0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


successfully generated models answers
SAE dataset saved to ./seperate models/Llama-3.1-SAE.csv
Number of processed sentences: 5000


In [268]:
aae_to_sae = pd.read_csv('./seperate models/Llama-3.1-SAE.csv')

In [246]:
#sae_labels = pd.read_csv('./seperate models/Llama-3.1-SAE-sentiment.csv')

In [247]:
#sae_labels['sae_labels'].unique()

In [248]:
#sae_labels.columns

In [249]:
#i_cant_assist = sae_labels.iloc[[50, 77, 690, 817, 1010, 1846, 2941]]

In [250]:
#i_cant_assist.head()

In [269]:
inverse_df = aae_to_sae.loc[~aae_to_sae.index.isin([50, 77, 690, 817, 1775, 1010, 1846, 2941])]

In [270]:
inverse_df.to_csv('./seperate models/llama-some-removed/Llama-3.1-SAE.csv', index=False)

In [271]:
inverse_df.shape

(4992, 2)

In [13]:
labeled_df.head()

,text,prompt,standard_american_english
0,Bitch cant get shit from me but bubble gum nd ...,[{'content': 'Following is a tweet extracted f...,"This woman can't get anything from me, but she..."
1,@islandboi_B yes that's what's up. Nothin like...,[{'content': 'Following is a tweet extracted f...,"@islandboi_B, yes that's what's going on. Ther..."
2,Mixed huh !? Those black ass knees and elbows ...,[{'content': 'Following is a tweet extracted f...,Mixed huh!? Those black knees and elbows will ...
3,The bul Mike James from @mavs ain't shit n he ...,[{'content': 'Following is a tweet extracted f...,Mike James from the Mavericks is completely us...
4,It took for a whole stranger to tell me he PRO...,[{'content': 'Following is a tweet extracted f...,It took a complete stranger to tell me how pro...


# Getting labels for SAE

In [14]:
df = pd.read_csv('./seperate models/Llama-3.1-SAE.csv')

In [15]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()


# Function to create prompts for sentiment analysis
def create_prompt(example):
    return  [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in Standard American English and identify the sentiment expressed by the author. Reply back in one word with just the sentiment, Positive, Negative, or Neutral only\n"},
    {"role": "user", "content": f"{example['standard_american_english']}"},
    {"role": "assistant", "content": "The sentiment expressed in this sentence is: "}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_prompt(example)})

def extract_sentiment(response):
    if "Positive" in response:
        return "Positive"
    elif "Negative"  in response:
        return "Negative"
    elif "Neutral" in response:
        return "Neutral"
    else:
        return "Unknown"

# Function to generate sentiments
def generate_sae_labels(examples):
    outputs = pipeline(
      examples["prompt"],
         max_new_tokens=5,
                do_sample=False,
        num_return_sequences=1
    )
    return {"sae_labels": [extract_sentiment(output[0]["generated_text"]) for output in outputs],
           "model_generation" : [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_sae_labels,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

output_path = './seperate models/Llama-3.1-SAE-sentiment.csv'
labeled_df[["standard_american_english", "sae_labels", "model_generation"]].to_csv(output_path, index=False)

print(f"SAE dataset saved to {output_path}")
print(f"Number of processed sentences: {len(labeled_df)}")

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Parameter 'function'=<function generate_sae_labels at 0x7fbc447a1af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


successfully generated models answers
SAE dataset saved to ./seperate models/Llama-3.1-SAE-sentiment.csv
Number of processed sentences: 5000


In [18]:
unknown = labeled_df[labeled_df["sae_labels"] == 'Unknown']

In [19]:
unknown.head()

,african_american_english,standard_american_english,prompt,sae_labels,model_generation
63,So at 2PM I was super tired. Now at almost 1AM...,"I was extremely exhausted at 2 PM, but now at ...",[{'content': 'Your task is to analyze the prov...,Unknown,Confused
159,Thank you Lord for waking me up this morning ....,"Thank you, Lord, for waking me up this morning...",[{'content': 'Your task is to analyze the prov...,Unknown,Mixed
985,"I feel like he dont even want to talk to me, ...",I feel like he doesn't even want to talk to me...,[{'content': 'Your task is to analyze the prov...,Unknown,Mixed
993,Back in the days when I was young um not a kid...,"When I was younger, I sometimes find myself si...",[{'content': 'Your task is to analyze the prov...,Unknown,Nostalgic
2498,Sometimes I wish I could rewind back to the su...,Sometimes I wish I could rewind back to the su...,[{'content': 'Your task is to analyze the prov...,Unknown,Nostalgic


In [2]:
import pandas as pd
sae_sentiment = pd.read_csv('./seperate models/Llama-3.1-SAE-sentiment.csv')

In [3]:
sae_sentiment = sae_sentiment.loc[~sae_sentiment.index.isin([50, 77, 690, 817, 1775, 1010, 1846, 2941])]

In [4]:
sae_sentiment.shape

(4992, 2)

In [5]:
sae_sentiment.to_csv('./seperate models/some-removed/Llama-3.1-SAE-sentiment.csv', index=False)

### changing the prompt, adding "No other words allowed in sentiment other than 'Positive', 'Negative' or 'Neutral'"

In [131]:
# Function to extract sentiment from model output
def extract_sentiment(response):
    if "Positive" in response:
        return "Positive"
    elif "Negative" in response:
        return "Negative"
    elif "Neutral" in response:
        return "Neutral"
    else:
        return "Unknown"
    
    
def generate_sentiment(sentence):
    outputs = pipeline(
        [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in Standard American English and identify the sentiment expressed by the author. The sentiment should be one of Positive, Negative, or Neutral. No other words are allowed in your answer."},
    {"role": "user", "content": f"{sentence}"},
    {"role": "assistant", "content": "The sentiment expressed in this sentence is: "}
],
        max_new_tokens=5,
        do_sample=False,
        num_return_sequences=1
    )
    return extract_sentiment(outputs[0]["generated_text"])

In [132]:
# Function to process a single row
def process_row_sentiment(row):
    return generate_sentiment(row["standard_american_english"])

In [133]:
test = labeled_df.copy()

In [134]:
mask = ~labeled_df["sae_labels"].isin(["Positive", "Negative", "Neutral"])
test.loc[mask, "sae_labels"] = labeled_df[mask].apply(process_row_sentiment, axis=1)

In [135]:
unknown = test[test["sae_labels"] == 'Unknown']

In [136]:
unknown.head()

,african_american_english,standard_american_english,prompt,sae_labels,model_generation


In [138]:
test[["standard_american_english", "sae_labels"]].to_csv("./seperate models/Llama-3.1-SAE-sentiment.csv", index=False)

## Back to AAE

In [139]:
df = pd.read_csv('./seperate models/Llama-3.1-SAE.csv')

In [141]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_aae_from_sae_prompt(example):
    return  [
    {"role": "system", "content": "You will be given a tweet in Standard American English. Your task is to convert the given tweet to African American English. Reply with just the translated sentence."},
    {"role": "user", "content": f"{example['standard_american_english']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset
dataset = dataset.map(lambda example: {"prompt": create_aae_from_sae_prompt(example)})

# Function to generate sentiments
def generate_aae_from_sae(examples):
    outputs = pipeline(
      examples["prompt"],
        max_new_tokens=100,
        do_sample=False,
        num_return_sequences=1
    )
    return {"AAE_from_SAE": [output[0]["generated_text"] for output in outputs]}

# Generate sentiments
dataset = dataset.map(
    generate_aae_from_sae,
    batched=True,
    batch_size=32
)

print("successfully generated models answers")

# Convert back to pandas DataFrame
labeled_df = dataset.to_pandas()

output_path = './seperate models/Llama-3.1-AAE_from_SAE.csv'
labeled_df[["standard_american_english", "AAE_from_SAE"]].to_csv(output_path, index=False)

In [276]:
aae_from_sae = pd.read_csv('./seperate models/Llama-3.1-AAE_from_SAE.csv')

In [277]:
aae_from_sae = aae_from_sae.loc[~aae_from_sae.index.isin([50, 77, 690, 817, 1775, 1010, 1846, 2941])]

In [278]:
aae_from_sae.to_csv('./seperate models/llama-some-removed/Llama-3.1-AAE_from_SAE.csv', index=False)

In [279]:
aae_from_sae.shape

(4992, 2)

## AAE_from_SAE labels

In [142]:
df = pd.read_csv('./seperate models/Llama-3.1-AAE_from_SAE.csv')

In [206]:
import datasets
from datasets import Dataset

datasets.logging.set_verbosity_info()

# Function to create prompts for sentiment analysis
def create_aae_from_sae_prompt(example):
    return  [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in African American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment."},
    {"role": "user", "content": f"{example['AAE_from_SAE']}"}
]

# Create a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Add prompts to the dataset

#dataset = dataset.map(lambda example: {"prompt": create_aae_from_sae_prompt(example)})

# Function to extract sentiment from model output
def extract_sentiment(output):
    if "Positive" in output:
        return "Positive"
    elif "Negative" in output:
        return "Negative"
    elif "Neutral" in output:
        return "Neutral"
    else:
        return "Unknown"


def generate_sentiment(sentence):
    outputs =  pipeline(
       [
    {"role": "system", "content": "Your task is to analyze the provided sentences written in African American English and identify the sentiment expressed by the author. The sentiment must be classified as Positive, Negative, or Neutral. Reply with just the sentiment."},
    {"role": "user", "content": f"{sentence}"}
],
        max_new_tokens=10,
        do_sample=False,
        num_return_sequences=1
    )
    return extract_sentiment(outputs[0]["generated_text"])

# Function to process a single row
def process_row_sentiment(row):
    return generate_sentiment(row["AAE_from_SAE"])

# Generate sentiments
#dataset = dataset.map(
#     generate_sentiment,
#     batched=True,
#     batch_size=32
# )

print("successfully generated models answers")

# Convert back to pandas DataFrame
#labeled_df = dataset.to_pandas()

# # Save the labeled dataset to a CSV file
#output_path = './seperate models/LLama3.1-8b-AAE_from_SAE-Sentiment.csv'
#labeled_df[["AAE_from_SAE", "AAE_from_SAE Sentiment"]].to_csv(output_path, index=False)

successfully generated models answers


In [144]:
# # Save the labeled dataset to a CSV file
# output_path = './seperate models/LLama3.1-8b-AAE_from_SAE-Sentiment.csv'
# labeled_df.rename(columns={"sentiment":"AAE_from_SAE Sentiment" })[["AAE_from_SAE", "AAE_from_SAE Sentiment"]].to_csv(output_path, index=False)

In [280]:
labeled_df = pd.read_csv('./seperate models/LLama3.1-8b-AAE_from_SAE-Sentiment.csv')

In [281]:
labeled_df['AAE_from_SAE Sentiment'].unique()

array(['Negative', 'Positive', 'Neutral', 'Unknown'], dtype=object)

In [282]:
test = labeled_df.copy()

In [283]:
test.columns

Index(['AAE_from_SAE', 'AAE_from_SAE Sentiment'], dtype='object')

In [284]:
test = test.loc[~test.index.isin([50, 77, 690, 817, 1010, 1775, 1846, 2941])]

In [285]:
mask = ~test["AAE_from_SAE Sentiment"].isin(["Positive", "Negative", "Neutral"])
test.loc[mask, "AAE_from_SAE Sentiment"] = test[mask].apply(process_row_sentiment, axis=1)

/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/local/home/furquanh/miniconda3/envs/augmentation-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [286]:
test['AAE_from_SAE Sentiment'].unique()

array(['Negative', 'Positive', 'Neutral'], dtype=object)

In [287]:
test[test["AAE_from_SAE Sentiment"] == 'Unknown']

,AAE_from_SAE,AAE_from_SAE Sentiment


In [288]:
test.to_csv('./seperate models/llama-some-removed/LLama3.1-8b-AAE_from_SAE-Sentiment.csv')

In [290]:
# Define the directory containing the CSV files and the output directory
input_directory = './Complete Dataset'  # Replace with your input directory path
output_directory = './Complete Dataset/some-removed/'

# Indices to be removed
indices_to_remove = [50, 77, 690, 817, 1010, 1846, 2941]

# Create the output directory if it doesn't exist
#os.makedirs(output_directory, exist_ok=True)

# Loop through every file in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv'):
        # Construct the full file path
        file_path = os.path.join(input_directory, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Remove the rows with the specified indices
        df_filtered = df.drop(indices_to_remove, errors='ignore')
        
        # Save the filtered DataFrame to the output directory with the same filename
        output_path = os.path.join(output_directory, filename)
        df_filtered.to_csv(output_path, index=False)

print("CSV files have been processed and saved in the 'some-removed' directory.")

CSV files have been processed and saved in the 'some-removed' directory.
